### 1) Import the Relevant Files & Libraries

In [1]:
import sib_main.sib_sub.sib_validation_main as svm
import sib_main.sib_sub.sib_validation_imports as svi

import copy
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.multipoint import MultiPoint
from shapely.ops import nearest_points
from datetime import datetime

ground_truth = gpd.read_file("E:/SIB/Individual_Models/model2/model2b/model2b_Highlands/3_validation/model2b_Highlands_Validationfiles/model2b_Highlands_GroundTruth.shp")
model_det = gpd.read_file("E:/SIB/Individual_Models/model2/model2b/model2b_Highlands/3_validation/model2b_Highlands_Validationfiles/model2b_Highlands_detect.shp")
VHRsamples = gpd.read_file("E:/SIB/Individual_Models/model2/model2b/model2b_Highlands/3_validation/model2b_Highlands_Validationfiles/model2b_Highlands_VHRValidation.shp")

export_path = "E:/SIB/Individual_Models/model2/model2b/model2b_Highlands/3_validation/"

### 2) Preprocess the Data

In [2]:
svi.confidence_filter(model_det)

90.0% of values above = 0.16771580000000003


0.16771580000000003

In [3]:
# Change 'geometry' column from 'MULTIPOINT' to 'POINT'
ground_truth = svi.multi_to_single(ground_truth)
model_det = svi.multi_to_single(model_det)

In [4]:
help(svm.preprocessing)

Help on function preprocessing in module sib_main.sib_sub.sib_validation_main:

preprocessing(ground_truth_points, model_detected_points, conf_percentile=10)
    Preprocessing GeoDataFrame before analysis.
    
    This function checks that the following criteria are met in the GeoDataFrame:
        a) checks geometry type
        b) filters the confidence values by a chosen percentile
        c) verifies the correct crs
        d) establishes a unique ID for each GeoDataFrame
        e) renames the relevant columns to a standard format
        f) drops any unrelated columns and attributes
    
    Parameters
    ----------
    ground_truth_points : GeoDataFrame
                          These points must be imported after being manually created.
                          They represent the actual ground truth class, as verified by the user.
    
    model_detected_points : GeoDataFrame
                            These points must be imported after having been converted from YOLO txt 

In [5]:
ground_truth, model_det = svm.preprocessing(ground_truth, model_det, conf_percentile=10)

Accepted Geometries: 'POINT'
Accepted Geometries: 'POINT'
90.0% of values above = 0.16771580000000003


In [6]:
print("Number of Ground Truth Points: {}".format(len(ground_truth)))
print("Number of Model Detected Points: {}".format(len(model_det)))

Number of Ground Truth Points: 1325
Number of Model Detected Points: 1402


### 3) Find the maximum value of detected points within a 2m radius of a ground truth point

In [7]:
n_ = svi.n_max(ground_truth, model_det)
n_

3

### 4) Run the body of the script

In [8]:
startTime = datetime.now()

global_tp = []
global_fp = []

for i in range(0, n_):

    # 1) CALCULATE NEAREST
    nearest_neighbour = svm.call_calculate_nearest(ground_truth, model_det)
    
    # 2) CALCUALTE DISTANCE
    nearest_neighbour = svm.calculate_distance(nearest_neighbour)
    
    # 3) CALCULATE TP
    nearest_neighbour, tp = svm.calculate_tp(nearest_neighbour)
    # Create a dictionary to append to the global list - will need it in dictionary format to create a df
    tp_dict = tp.to_dict('list')
    try:
        global_tp.append(tp_dict)
    except AttributeError:
        print("No more True Positives to append")
        pass
 
    # 4) CALCULATE FP
    nearest_neighbour, fp = svm.calculate_fp(nearest_neighbour)
    fp_dict = fp.to_dict('list')
    try:
        global_fp.append(fp_dict)
    except AttributeError:
        print("No more False Positives to append")
        pass
         
    # 5) DROP INDICES
        # DROP TP INDICES
    model_det = svi.drop_indices(model_det, tp, outcome_value='true')
    
        # DROP FP INDICES
    model_det = svi.drop_indices(model_det, fp, outcome_value='false')
    
        # DROP GROUND TRUTH INDICES
    ground_truth = svi.drop_indices(ground_truth, tp, outcome_value='ground')
    
# 6) CALCULATE THE FALSE POSITIVES OUTSIDE THE 2M RADIUS
fp2 = svi.extract_points_outside_2m(ground_truth, model_det)
fp2_dict = fp2.to_dict('list')
try:
    global_fp.append(fp2_dict)
except AttributeError:
    pass

# 7) CREATE GEODATAFRAMES FOR TRUE POSITIVES AND FALSE POSITIVES
global_tp_df = svi.create_global_GeoDataFrame(global_tp)
global_fp_df = svi.create_global_GeoDataFrame(global_fp)
print("TRUE POSITIVEs GEODATAFRAME CONSTRUCTED")
print("FALSE POSITIVEs GEODATAFRAME CONSTRUCTED\n")

# 8) CALCULATE FN
# Drop all False Positives now from the Ground Truth points. 
ground_truth = svi.drop_indices(ground_truth, global_fp_df, outcome_value='ground')
global_fn_df = svm.calculate_fn(ground_truth, model_det)
print("FALSE NEGATIVES GEODATAFRAME CONSTRUCTED\n")

# 9) CREATE_VALIDATION_DATAFRAME
VHRsamples = svm.create_validation_dataframe(VHRsamples, global_tp_df, global_fp_df, global_fn_df)
print("VHRVALIDATION GEODATAFRAME CONSTRUCTED")


print(datetime.now() - startTime)

C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Initial True Positives, Count: 912
Initial False Positives, Count: 198
Length of 'model_det' after TP rows dropped: 490

Length of 'model_det' after FP rows dropped: 292

Length of 'Ground Truth points' after TP rows dropped: 413



C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Initial True Positives, Count: 17
Initial False Positives, Count: 9
Length of 'model_det' after TP rows dropped: 275

Length of 'model_det' after FP rows dropped: 266

Length of 'Ground Truth points' after TP rows dropped: 396



C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Initial True Positives, Count: 0
There were no incorrect classifications (FP1) detected on this pass.
Length of 'model_det' after TP rows dropped: 266

Length of 'model_det' after FP rows dropped: 266

Length of 'Ground Truth points' after TP rows dropped: 396



C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


TRUE POSITIVEs GEODATAFRAME CONSTRUCTED
FALSE POSITIVEs GEODATAFRAME CONSTRUCTED

Length of 'Ground Truth points' after FP rows dropped: 211

FALSE NEGATIVES GEODATAFRAME CONSTRUCTED

VHRVALIDATION GEODATAFRAME CONSTRUCTED
0:04:16.396741


In [9]:
svm.export_GeoDataFrame(export_path, VHRsamples, file_name='VHRSamples', extension='.shp')
svm.export_GeoDataFrame(export_path, VHRsamples, file_name='VHRSamples', extension='.csv')
svm.export_GeoDataFrame(export_path, global_tp_df, file_name='TP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fp_df, file_name='FP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fn_df, file_name='FN', extension='.shp')
svm.export_GeoDataFrame(export_path, ground_truth, file_name='ground_truth_OUTPUT', extension='.shp')
svm.export_GeoDataFrame(export_path, model_det, file_name='model_det_OUTPUT', extension='.shp')

Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
